In [1]:
import os
import sys

nlp_path = os.path.abspath('../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

In [2]:
import re
def regex_or(*items):
    return '(?:' + '|'.join(items) + ')'


punctChars = r"['\"“”‘’.?!…,:;]"
#punctSeq   = punctChars+"+"	#'anthem'. => ' anthem '.
punctSeq   = r"['\"“”‘’]+|[.?!,…]+|[:;]+"	#'anthem'. => ' anthem ' .
entity     = r"&(?:amp|lt|gt|quot);"
#  URLs
urlStart1  = r"(?:https?://|\bwww\.)"
commonTLDs = r"(?:com|org|edu|gov|net|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|pro|tel|travel|xxx)"
ccTLDs	 = r"(?:ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|" + \
r"bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|" + \
r"er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|" + \
r"hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|" + \
r"lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|" + \
r"nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|sk|" + \
r"sl|sm|sn|so|sr|ss|st|su|sv|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|" + \
r"va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|za|zm|zw)"	#TODO: remove obscure country domains?
urlStart2  = r"\b(?:[A-Za-z\d-])+(?:\.[A-Za-z0-9]+){0,3}\." + regex_or(commonTLDs, ccTLDs) + r"(?:\."+ccTLDs+r")?(?=\W|$)"
urlBody    = r"(?:[^\.\s<>][^\s<>]*?)?"
urlExtraCrapBeforeEnd = regex_or(punctChars, entity) + "+?"
urlEnd     = r"(?:\.\.+|[<>]|\s|$)"
url        = regex_or(urlStart1, urlStart2) + urlBody + "(?=(?:"+urlExtraCrapBeforeEnd+")?"+urlEnd+")"


Bound = r"(?:\W|^|$)"
Email = regex_or("(?<=(?:\W))", "(?<=(?:^))") + r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}(?=" +Bound+")"
AtMention = "[@＠][a-zA-Z0-9_]+"

In [3]:
tw = "@huyen I am goinnt to school https://duo.com.vn"
s = re.sub(regex_or(url,Email), '', tw)

In [4]:
print(s)

@huyen I am goinnt to school 


In [5]:
from bert_score import BERTScorer
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from lsh import LSH
from fast_lexrank import Lexrank
from numpy.linalg import norm
import time, emoji, string
from joblib import Parallel, delayed
# hide the loading messages
from utils import tokenizeRawTweetText
import scipy
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings; warnings.simplefilter('ignore')

In [6]:
# read data
# data = pd.read_csv('/home/ehoang/hnt/data/processed_travel_ban_withURL.csv')
# data = pd.read_csv('/home/ehoang/git/python/tweet_classification/data/CrisisLexT6/2012_Sandy_Hurricane/2012_Sandy_Hurricane-ontopic_offtopic.csv')
data = pd.read_csv('/home/ehoang/hnt/data/travel_ban.csv')
print(data.head())
print(data.shape)

                   id                                              Tweet
0  824941360449015808  RT @nancykric: Emergency Rally Against Trump's...
1  824941360449015808  RT @nancykric: Emergency Rally Against Trump's...
2  824941360449015808  RT @nancykric: Emergency Rally Against Trump's...
3  824941519857610752  RT @tponews: Theresa May has not apologized to...
4  824941616314122240  RT @TheDemCoalition: Trump's Immigration Ban E...
(123387, 2)


In [4]:
data.columns = ['TweetId', 'Tweet', 'label']
data = data[data['label'] == 'on-topic']
data.shape

(6138, 3)

In [7]:
data = data.reset_index(drop=True)

In [8]:
for tweet in list(data.iloc[0:15]['Tweet']):
    print(tweet)

RT @nancykric: Emergency Rally Against Trump's Muslim Travel Ban in NYC, 1/25 at 5 p.m. https://t.co/BNC3Gz7NVb

RT @nancykric: Emergency Rally Against Trump's Muslim Travel Ban in NYC, 1/25 at 5 p.m. https://t.co/BNC3Gz7NVb

RT @nancykric: Emergency Rally Against Trump's Muslim Travel Ban in NYC, 1/25 at 5 p.m. https://t.co/BNC3Gz7NVb

RT @tponews: Theresa May has not apologized to Trump for insulting him. If she fails to do that today, Trump should just send her back to B…

RT @TheDemCoalition: Trump's Immigration Ban Excludes Countries with Business Ties https://t.co/8dtBQve4op via @bbgvisualdata #DemocracyFor…

RT @nytimes: Trump's immigration order expands the definition of "criminal" https://t.co/fNpFTyQTwh https://t.co/AB8gSJdbyV

ALERT: Senator John McCain Threatens Action On President Trump If He Does This https://t.co/5zhD1FjisZ

@TrumpMyPres @realDonaldTrump @LeahR77 @Lrihendry @TruthFeedNews  Kiva still distracted while Trump gets on with people's business.

RT @BillyBaldwi

In [9]:
data['Tweet'] = data['Tweet'].apply(lambda x: ' '.join(tokenizeRawTweetText(x)))

In [10]:
for tweet in list(data.iloc[0:15]['Tweet']):
    print(tweet)

RT TWEETMENTION : Emergency Rally Against Trump's Muslim Travel Ban in NYC , 1/25 at 5 p.m. HTTPURL
RT TWEETMENTION : Emergency Rally Against Trump's Muslim Travel Ban in NYC , 1/25 at 5 p.m. HTTPURL
RT TWEETMENTION : Emergency Rally Against Trump's Muslim Travel Ban in NYC , 1/25 at 5 p.m. HTTPURL
RT TWEETMENTION : Theresa May has not apologized to Trump for insulting him . If she fails to do that today , Trump should just send her back to B …
RT TWEETMENTION : Trump's Immigration Ban Excludes Countries with Business Ties HTTPURL via TWEETMENTION #DemocracyFor …
RT TWEETMENTION : Trump's immigration order expands the definition of " criminal " HTTPURL HTTPURL
ALERT : Senator John McCain Threatens Action On President Trump If He Does This HTTPURL
TWEETMENTION TWEETMENTION TWEETMENTION TWEETMENTION TWEETMENTION Kiva still distracted while Trump gets on with people's business .
RT TWEETMENTION : TY TWEETMENTION for bailing on GMB & TWEETMENTION today . Piers Morgan drank the Trump Kool A

In [11]:
# remove rt, @USER, @URL, emoji
data['Tweet'] = data['Tweet'].apply(lambda x: x.replace('TWEETMENTION', "").
                                    replace("EMAILADDRESS", "").replace('HTTPURL', ''))
data['Tweet'] = data['Tweet'].apply(lambda x: x.lower().strip())
data['Tweet'] = data['Tweet'].apply(lambda x: re.sub("^ ?(rt ?)+", "", x))                              
data['Tweet'] = data['Tweet'].apply(lambda x: re.sub('^( ?: ?)', '', x))
data['Tweet'] = data['Tweet'].apply(lambda x: re.sub("  +", " ", x))
data['Tweet'] = data['Tweet'].apply(lambda x: ''.join(c for c in x if c not in emoji.UNICODE_EMOJI).strip())

In [12]:
for tweet in list(data.iloc[0:15]['Tweet']):
    print(tweet)

emergency rally against trump's muslim travel ban in nyc , 1/25 at 5 p.m.
emergency rally against trump's muslim travel ban in nyc , 1/25 at 5 p.m.
emergency rally against trump's muslim travel ban in nyc , 1/25 at 5 p.m.
theresa may has not apologized to trump for insulting him . if she fails to do that today , trump should just send her back to b …
trump's immigration ban excludes countries with business ties via #democracyfor …
trump's immigration order expands the definition of " criminal "
alert : senator john mccain threatens action on president trump if he does this
kiva still distracted while trump gets on with people's business .
ty for bailing on gmb & today . piers morgan drank the trump kool aid & is a vocal opponent o …
#trump to sign eo temporary ban suspending visas for syria & six other me , african countries #buildthewall  …
did we have a moral obligation to stop hitler ? if so we have a moral obligation to stop trump .
are these people just now getting radicalized by 

In [13]:
# data = data.iloc[0:5200]

In [14]:
# remove stopwords, punctuation
stopWords = stopwords.words('english')
# data['Tweet1'] = data['Tweet'].apply(lambda x: re.sub(Email, '', x))
# data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub(url, '', x))
# data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub(AtMention, '', x))
# data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub("^ ?(rt ?)+", "", x))                              
# data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub('^( ?: ?)', '', x))
# data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub("  +", " ", x))

data['Tweet1'] = data['Tweet'].apply(lambda x: ' '.join(y for y in x.split(" ") if y not in stopWords))

data['Tweet1'] = data['Tweet1'].apply(lambda x: x.translate(str.maketrans('', '',  string.punctuation)))
data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub('“|…|’|‘|”|—|→', "", x))
data['Tweet1'] = data['Tweet1'].apply(lambda x: re.sub(' +', ' ',x).strip())

# remove tweets #unique words less than haft of length
data['uniWPercent'] = data['Tweet1'].apply(lambda x: 0 if len(set(x.split(" ")))/len(x.split(" ")) <= 0.5 else len(x.split(" ")))
data = data[data['uniWPercent']!=0]
# # remove tweets with lengths < 3, duplicates
while data['uniWPercent'].min() <=2:
    data = data[data['uniWPercent'] >2]
    data['uniWPercent'] = data['Tweet1'].apply(lambda x: 0 if len(set(x.split(" ")))/len(x.split(" ")) <= 0.5 else len(x.split(" ")))
# # # remove duplicates
data.drop_duplicates(subset=['Tweet1'], keep='first', inplace = True)
print(data.head())

                   id                                              Tweet  \
0  824941360449015808  emergency rally against trump's muslim travel ...   
3  824941519857610752  theresa may has not apologized to trump for in...   
4  824941616314122240  trump's immigration ban excludes countries wit...   
5  824942056741167105  trump's immigration order expands the definiti...   
6  824942966875774976  alert : senator john mccain threatens action o...   

                                              Tweet1  uniWPercent  
0  emergency rally trumps muslim travel ban nyc 1...           10  
3  theresa may apologized trump insulting fails t...           11  
4  trumps immigration ban excludes countries busi...            9  
5  trumps immigration order expands definition cr...            6  
6  alert senator john mccain threatens action pre...            8  


In [15]:
data.shape

(105175, 4)

In [16]:
remained_index = data.index

In [17]:
remained_index

Int64Index([     0,      3,      4,      5,      6,      7,      8,      9,
                10,     11,
            ...
            123376, 123377, 123378, 123379, 123380, 123381, 123382, 123383,
            123385, 123386],
           dtype='int64', length=105175)

In [18]:
data = data.reset_index(drop=True)

In [19]:
data.shape

(105175, 4)

In [21]:
# data = data.iloc[0:5000]

In [22]:
# data['Tweet'] = data['Tweet'].apply(lambda x: x.lower())

In [23]:
str(data.iloc[1]['Tweet'])

'neighborly duties . arrives to the rescue sporting some spelunking equipment #sandy @ 300 squad httpurl'

In [18]:
class Lexrank:
    """
    lexrank model combined with lsh & cosine similarity
    """

    def __init__(self, data):
        self.data = data
        self.graph = {}
        self.matrix = None
        self.scores = None
        
        
    def build_graph_bertscore_file(self, input_file, sim_thres=0.3):        
        count = 0
        with open(input_file, 'r') as f:
            for i, line in enumerate(f):
                
                content = line.split(",")
                idx = int(content[0])
                if idx not in remained_index:
                    continue
                if content[1][1:-1]=="":
                    continue
                print(content[1][1:-1])
                print(content[2][1:-2])
                neighbors = [int(x) for x in content[1][1:-1].split(',')]
            
                sim_scores = [float(y) for y in content[2][1:-2].split(" ") if y!='']
                
                for j, neighbor_idx in enumerate(neighbors):
                    if idx + neighbor_idx+1 not in remained_index or sim_scores[j] <sim_thres:
                        continue

                    if idx not in self.graph:
                        self.graph[idx] = {}
                    self.graph[idx][idx + neighbor_idx+1] = sim_scores[j]
                    if idx+neighbor_idx+1 not in self.graph:
                        self.graph[idx + neighbor_idx+1] = {}
                    self.graph[idx + neighbor_idx+1][idx] = sim_scores[j]
#                     print(idx, idx+j+1, score)
                if i%200 == 0:
                    print("Line: ", i, idx)
        print("Done, ", i, count)
           

    def build_graph_bertscore(self, input_file, sim_thres=0.3):        
        count = 0
        with open(input_file, 'r') as f:
            for i, line in enumerate(f):
                
                
                content = line[0:line.index('[')].split(',')
                idx = int(content[0])
#                 if idx not in remained_index:
#                     continue
                if (line[line.index('[')+1:len(line)-2]==""):
                    print("..................................")
                    continue
                try:
                    sim_scores = [float(x) for x in line[line.index('[')+1:len(line)-2].split(',')]
                except Exception:
                    print(line)
                
                for j, score in enumerate(sim_scores):
                    if score <=sim_thres:
                        continue
#                     if (idx+j+1 not in remained_index):
#                         continue
                    if idx not in self.graph:
                        self.graph[idx] = {}
                    self.graph[idx][idx+j+1] = score
                    if idx+j+1 not in self.graph:
                        self.graph[idx+j+1] = {}
                    self.graph[idx+j+1][idx] = score
#                     print(idx, idx+j+1, score)
                if i%200 == 0:
                    print("Line: ", i, idx)
        print("Done, ", i, count)

    def build_graph_cosine1(self, sim_thres = 0.3, batch_size = 1000):
        for i in range(0, data.shape[0], 1):
            cosine = cosine_similarity(self.data[i], self.data)[0]
            idx = np.where(cosine>sim_thres)[0]
            for id in idx:
                if id==i:
                    continue
                if i not in self.graph:
                    self.graph[i] = {}
                self.graph[i][id] = cosine[id]
                
            if i%1000 == 0:
                print("Line: ", i)
                    
    def build_graph_cosine(self, sim_thres = 0.3, batch_size = 1000):
        for i in range(0, data.shape[0], batch_size):
            j = i
            if i + batch_size>data.shape[0]:
                sents = self.data[i:data.shape[0]]

            else:
                sents = self.data[i:i+batch_size]
            
            for j in range(i, data.shape[0], batch_size):
#                 if j == 0:
#                     continue
                rightBound = j+batch_size
                if rightBound > data.shape[0]:
                    rightBound = data.shape[0]

                sents1 = self.data[j: rightBound]  # get list of sentVecs

                num = np.dot(sents, sents1.T)
                if scipy.sparse.issparse(sents):
                    magnitude = norm(sents.toarray(), axis=1)
                    magnitude1 = norm(sents1.toarray(), axis=1)
                else:
                    magnitude = norm(sents, axis=1)
                    magnitude1 = norm(sents1, axis=1)
                den = np.dot(magnitude.reshape(-1, 1), magnitude1.T.reshape(1, -1))

                cosine_matrix = np.array(num / den)
                indices = np.where(cosine_matrix > sim_thres)
#                 print("matrix:", cosine_matrix.shape)
                if len(indices[0]) == 0:
                    continue
                try:
                    for row, col in zip(indices[0], indices[1]):
                        if i+row != j+col:  # ignore self-links
        #                     matrix_indices.append([b[row], b[col]])
        #                     weights.append(cosine_matrix[row][col])
        #                     weights.append(1)

                            if i+row not in self.graph:
                                self.graph[i+row] = {}
                            if j+col not in self.graph:
                                self.graph[j+col] = {}

                            self.graph[i+row][j+col] = cosine_matrix[row][col]
                            self.graph[j+col][i+row] = cosine_matrix[row][col]
    #                         print(i, j, i+row, j+col, cosine_matrix[row][col])
                except Exception:
                    print("Error:", i, j, row, col, cosine_matrix.shape)

    # using pagerank pagekage
    def page_rank(self, damping_factor=0.85):
        pr = pagerank(self.matrix, p=damping_factor)
        self.scores = pr

    def train(self, lexrank_iter=100, damping_factor=0.85):
        n = self.data.shape[0]

        # for each node: compute sum of weights of adjacent nodes
        sum_weights = {}
        for sent, adj in self.graph.items():
            sum_weights[sent] = sum(adj.values())

        self.scores = [1 / n] * n  # initialize pagerank scores
        try:

            for iter in range(lexrank_iter):
                if iter % 10 == 0:
                    print("Iteration: {}".format(iter))
                for sent, adjs in self.graph.items():
                    score = 0
                    for adj, value in adjs.items():
                        score += self.scores[adj] * value / sum_weights[adj]
                    self.scores[sent] = (1 - damping_factor)/n +damping_factor * score
        except Exception:
            print(sent)
            print(adj)

    def extract_summary(self, n_sents=10, cosine_thres=0.5, max_sent=100):

        sentIds = []
        sentScores = np.array(self.scores.copy())

        print("Extracting sentences....")
        # get #max_sent maximal scores along with its indices
        print("Sent scores: {}".format(len(sentScores)))

        indices = np.argpartition(sentScores, -max_sent)[-max_sent:]
        values = sentScores[indices]
        max_index_value = {key: value for key, value in zip(indices, values)}
        max_index_value = sorted(max_index_value.items(), key=lambda x: (x[1], x[0]))

        i = 0
        while i < n_sents:
            index, value = max_index_value.pop()
            if index not in self.graph:
                print("Sent {} not in graph".format(index))
                continue
            assign = 1
            # iterate selected sentences
            for idx in sentIds:
                # if new index is not an ajdacent node of the selected one
                if idx not in self.graph[index]:
                    continue
                similarity = self.graph[index][idx]
                if similarity > cosine_thres:
                    print("Sent {} is similar to a {}: {}".format(index, idx, similarity))
                    assign = 0
                    break
            if assign == 1:
#                 print(i, ", ", 'TweetId: ', self.data.iloc[index]['Id'], ": ", self.data.iloc[index]['Tweet'])
                print("selected one: {}, {}".format(index, value))
                sentIds.append(index)
                i += 1
        return sentIds

In [19]:
lex = Lexrank(data)

In [20]:
lex.build_graph_bertscore(input_file = "/home/ehoang/hnt/data/files/bertscore_sandy_noURLs_tfidf.txt", sim_thres=0.15)

Line:  0 4662
Line:  200 5060
Line:  400 338
Line:  600 3016
Line:  800 4038
Line:  1000 3031
Line:  1200 4076
Line:  1400 4482
Line:  1600 4991
Line:  1800 3434
Line:  2000 4592
Line:  2200 2762
Line:  2400 1404
Line:  2600 3874
Line:  2800 3532
Line:  3000 3930
Line:  3200 2483
Line:  3400 2500
Line:  3600 3643
Line:  3800 2189
Line:  4000 1163
Line:  4200 1181
Line:  4400 2621
Line:  4600 872
Line:  4800 193
Line:  5000 578
Line:  5200 995
Done,  5348 0


In [21]:
len(lex.graph)

5178

In [22]:
lex.train(lexrank_iter=100, damping_factor=0.85)

Iteration: 0
Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90


In [23]:
bertscore_dict = {}
for key, value in lex.graph.items():
    bertscore_dict[key] = len(value)


In [24]:
bertscore_dict = {k: v for k, v in sorted(bertscore_dict.items(), key=lambda item: item[1], reverse = True)}

In [25]:
bertscore_dict

{97: 3090,
 1513: 1253,
 1395: 1223,
 1315: 1195,
 245: 1000,
 2767: 996,
 3161: 991,
 6: 979,
 740: 967,
 1129: 924,
 1126: 918,
 1688: 892,
 1310: 869,
 3266: 864,
 1865: 823,
 1628: 734,
 1835: 719,
 1856: 713,
 1719: 652,
 2650: 616,
 1524: 525,
 385: 514,
 923: 508,
 1023: 495,
 1915: 486,
 2966: 471,
 166: 470,
 2358: 447,
 4068: 446,
 3051: 424,
 1022: 410,
 3178: 409,
 2715: 402,
 2667: 399,
 3298: 385,
 3065: 384,
 333: 384,
 324: 382,
 189: 381,
 1279: 379,
 2944: 373,
 2856: 370,
 1337: 360,
 3332: 356,
 2289: 356,
 630: 350,
 2159: 339,
 2014: 333,
 1663: 330,
 1983: 329,
 1498: 327,
 729: 324,
 1474: 322,
 524: 321,
 314: 320,
 1842: 313,
 1470: 311,
 1792: 302,
 2082: 302,
 243: 296,
 329: 296,
 1419: 294,
 1203: 294,
 4410: 292,
 2649: 291,
 1790: 291,
 2021: 287,
 1560: 279,
 1812: 276,
 3307: 273,
 1896: 270,
 3303: 259,
 1169: 254,
 2928: 253,
 1052: 253,
 1401: 252,
 3263: 251,
 1397: 248,
 1475: 246,
 3668: 245,
 297: 242,
 1583: 239,
 2709: 238,
 2516: 234,
 2190: 

In [26]:
count = 0
selected = []
for key, value in bertscore_dict.items():
    
    if count>0:
        added = True
        for k in selected:
            if k in lex.graph[key]:
                if lex.graph[key][k]>0.25:
                    added = False
                    break
        
        if added==True:
            selected.append(key)
            count+=1
            print(count, ".", key, str(data.iloc[key]['Tweet']))
    else:
        selected.append(key)
        count+=1
        print(key, str(data.iloc[key]['Tweet']))
            
    if count > 20: 
        break

97 attention everyone i lost power
2 . 1513 hurricane sandy map shows storm's path towards u.s. northeast
3 . 1395 i'm scared about this hurricane ...
4 . 1315 this hurricane is going to be a joke
5 . 2767 these hurricane sandy tweets suck .
6 . 6 debating going home in prep for #sandy
7 . 740 beach is crazy right now
8 . 1865 google event cancelled due to hurricane
9 . 1719 i want to be kissed in a hurricane .. ?
10 . 1524 baby delivered in n.j. hospital truck during hurricane sandy
11 . 3178 wait there's a hurricane coming ?
12 . 3065 playlist to narrate what happens during the hurricane
13 . 324 what people in new york do during a hurricane
14 . 2159 i blame obama for hurricane sandy .
15 . 1983 praying for everyone in the path of hurricane sandy !
16 . 1498 no classes tomorrow . clutch hurricane !!!
17 . 524 map : monitor flood gauges live . #sandy
18 . 1470 these hurricane photos look awful . please stay safe all of you .
19 . 329 ah it's not so bad outside .
20 . 1203 done with t

In [35]:
sentIds = lex.extract_summary(n_sents=20, cosine_thres=0.25, max_sent=1500)

Extracting sentences....
Sent scores: 5350
selected one: 97, 0.021868332894725055
selected one: 6, 0.006277582555119697
selected one: 1513, 0.005977756978749937
selected one: 1395, 0.005853658760571331
selected one: 1315, 0.0056334146301175924
selected one: 2767, 0.005083392886882797
Sent 245 is similar to a 97: 0.2845384
selected one: 740, 0.004846867675394064
Sent 3161 is similar to a 6: 0.33088303
Sent 1129 is similar to a 1395: 0.43724263
selected one: 1865, 0.00406436696484457
Sent 1310 is similar to a 97: 0.31512138
Sent 1688 is similar to a 97: 0.25484043
Sent 1126 is similar to a 97: 0.337744
Sent 3266 is similar to a 97: 0.28127363
Sent 1856 is similar to a 1513: 0.43856743
Sent 1628 is similar to a 97: 0.25422952
Sent 1835 is similar to a 1395: 0.4651078
selected one: 1719, 0.003033823155923862
Sent 166 is similar to a 97: 0.2585594
Sent 923 is similar to a 97: 0.5842119
Sent 385 is similar to a 97: 0.4744008
Sent 2650 is similar to a 97: 0.34836107
selected one: 324, 0.00241

In [36]:
for i, idx in enumerate(sentIds):
    print(i+1,". ", idx, len(lex.graph[idx]), data.iloc[idx]['Tweet'])

1 .  97 3090 attention everyone i lost power
2 .  6 979 debating going home in prep for #sandy
3 .  1513 1253 hurricane sandy map shows storm's path towards u.s. northeast
4 .  1395 1223 i'm scared about this hurricane ...
5 .  1315 1195 this hurricane is going to be a joke
6 .  2767 996 these hurricane sandy tweets suck .
7 .  740 967 beach is crazy right now
8 .  1865 823 google event cancelled due to hurricane
9 .  1719 652 i want to be kissed in a hurricane .. ?
10 .  324 382 what people in new york do during a hurricane
11 .  1524 525 baby delivered in n.j. hospital truck during hurricane sandy
12 .  2159 339 i blame obama for hurricane sandy .
13 .  2667 399 turn up for hurricane sandy ??
14 .  3332 356 when is the hurricane hitting us ?
15 .  1498 327 no classes tomorrow . clutch hurricane !!!
16 .  524 321 map : monitor flood gauges live . #sandy
17 .  3065 384 playlist to narrate what happens during the hurricane
18 .  1983 329 praying for everyone in the path of hurricane san

# Tfidf

In [25]:
#extract tfidf vector
tfidf = TfidfVectorizer()
tfidfData = tfidf.fit_transform(data['Tweet1'])
print(tfidfData.shape)

(5350, 8255)


In [33]:
lex_tfidf = Lexrank(tfidfData)

In [34]:
lex_tfidf.build_graph_cosine(sim_thres = 0.3, batch_size = 1000)

In [46]:
scores = {key:len(value) for key, value in lex_tfidf.graph.items()}
scores = {key: value for key, value in sorted(scores.items(), key=lambda item: item[1],reverse = True)}

In [47]:
scores

{3755: 111,
 5324: 110,
 4485: 92,
 5029: 90,
 333: 84,
 2741: 83,
 4422: 83,
 4307: 81,
 324: 75,
 3668: 75,
 4027: 75,
 1792: 71,
 3509: 70,
 1560: 69,
 3274: 67,
 2021: 65,
 1860: 58,
 3731: 56,
 5085: 56,
 179: 55,
 2026: 55,
 3316: 55,
 1446: 55,
 5307: 54,
 1620: 52,
 2462: 52,
 1820: 51,
 4275: 51,
 4774: 50,
 4758: 49,
 4265: 49,
 1935: 47,
 2598: 47,
 3778: 47,
 1644: 46,
 3104: 46,
 60: 44,
 87: 44,
 2884: 44,
 1983: 44,
 5335: 43,
 4911: 42,
 1313: 41,
 2401: 41,
 3529: 40,
 4886: 40,
 1424: 39,
 3313: 39,
 1583: 38,
 1474: 38,
 2558: 38,
 2080: 38,
 2381: 38,
 3236: 38,
 380: 37,
 2003: 37,
 4327: 37,
 4370: 37,
 1195: 36,
 3064: 36,
 3264: 36,
 4253: 36,
 145: 35,
 195: 35,
 291: 35,
 354: 35,
 378: 35,
 394: 35,
 403: 35,
 600: 35,
 631: 35,
 664: 35,
 673: 35,
 726: 35,
 758: 35,
 805: 35,
 980: 35,
 993: 35,
 1020: 35,
 1028: 35,
 1073: 35,
 1475: 35,
 2613: 35,
 3968: 35,
 914: 34,
 2370: 34,
 3263: 34,
 3657: 34,
 3867: 34,
 5070: 34,
 5265: 34,
 579: 33,
 1152: 33,
 

In [49]:
count = 0
for key, value in scores.items():
    print(str(data.iloc[key]['Tweet']), value)
    count+=1
    if count>10: break

it was a hurricane in new york . ?????? 111
hurricane sandy for east coast 110
our thoughts are with everyone affected by hurricane sandy . stay safe everyone ! 92
praying for everyone who were affected by hurricane sandy !!! stay safe ! 90
i hope everyone is safe 84
my thoughts and prayers go out to all those affected by hurricane sandy . everyone stay safe out there . 83
my prayers go out to everyone affected by hurricane sandy .. stay safe ! 83
everyone being affected by the hurricane stay safe #praying 81
what people in new york do during a hurricane 75
everyone be safe in this hurricane ! 75
thoughts and prayers go out to all those affected by hurricane sandy . \hope that everyone is safe .. #hurricanesandy 75


In [35]:
lex_tfidf.train(lexrank_iter=100, damping_factor=0.85)

Iteration: 0
Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90


In [39]:
sentIds_cosine = lex_tfidf.extract_summary(n_sents=20, cosine_thres=0.3, max_sent=1000)

Extracting sentences....
Sent scores: 5350
selected one: 5324, 0.0028773535870559564
selected one: 3755, 0.002545551540132283
Sent 324 is similar to a 3755: 0.8254596281092088
selected one: 4886, 0.0013054529579528517
selected one: 333, 0.001286314680106863
Sent 4485 is similar to a 333: 0.5556057134541129
Sent 5029 is similar to a 333: 0.4548545016091528
Sent 3668 is similar to a 333: 0.7918207856407993
Sent 4307 is similar to a 333: 0.4608187662633855
Sent 2741 is similar to a 333: 0.3780173765424418
Sent 2884 is similar to a 333: 0.38502527046381985
Sent 4422 is similar to a 333: 0.4205533486230221
Sent 3274 is similar to a 5324: 0.6071776255846434
selected one: 933, 0.0010965197650889221
Sent 1560 is similar to a 5324: 0.6269714011260105
Sent 1792 is similar to a 5324: 0.564147559607847
Sent 4027 is similar to a 333: 0.5401455044631046
selected one: 5265, 0.0010105173522199629
selected one: 3104, 0.0009916927818131437
selected one: 3316, 0.0009891015197993108
Sent 2021 is similar t

In [40]:
for idx in sentIds_cosine:
    print(idx, len(lex_tfidf.graph[idx]), lex_tfidf.scores[idx])

5324 110 0.0028773535870559564
3755 111 0.002545551540132283
4886 40 0.0013054529579528517
333 84 0.001286314680106863
933 28 0.0010965197650889221
5265 34 0.0010105173522199629
3104 46 0.0009916927818131437
3316 55 0.0009891015197993108
1620 52 0.0009313422108900741
1395 31 0.00091183076015459
3448 31 0.0008580294427979393
5028 32 0.0008554177619808341
1832 27 0.0008323552490153656
5085 56 0.0008133138829930621
1244 29 0.0007881230422386743
3565 15 0.0007822049464827706
4830 25 0.0007728817993875358
3280 29 0.0007692841070275546
2562 29 0.000738897847287281
4926 21 0.0007272368352956854


In [41]:
for i, idx in enumerate(sentIds_cosine):
    print(i+1, ". ", idx, data.iloc[idx]['Tweet'])

1 .  5324 hurricane sandy for east coast
2 .  3755 it was a hurricane in new york . ??????
3 .  4886 the hurricane #sandy is a bitch
4 .  333 i hope everyone is safe
5 .  933 hurricane sandy on its way !!!
6 .  5265 there's a hurricane coming ..? where have i been ..
7 .  3104 really ? hurricane sandy has a twitter .
8 .  3316 praying for those in hurricane sandy
9 .  1620 hurricane sandy tweets are too funny ??
10 .  1395 i'm scared about this hurricane ...
11 .  3448 i want the hurricane to come so we don't have school ..
12 .  5028 why would you name a hurricane sandy ?
13 .  1832 how you can help after hurricane sandy\
14 .  5085 my thoughts and prayers go out to those affected by the hurricane .
15 .  1244 fuck you sandy . fuck you insomnia . & fuck you twitter .
16 .  3565 hurricane sandy : view from above
17 .  4830 why didn't the hurricane blow you away ? :)))))))))
18 .  3280 no school tomorrow & tuesday . #hurricane
19 .  2562 hurricane sandy is fucking my day up !
20 .  4926

## bert sentence + cosine

In [50]:
# extracting bertsen embeddings
from sentence_transformers import SentenceTransformer
import torch

In [51]:
data.shape

(5350, 5)

In [52]:
batch_size = 1000
modelSent = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda:1')
embeddings = np.empty((0, 768))

leftBound = 0
with torch.no_grad():
    while  leftBound < data.shape[0]:
        rightBound = leftBound+batch_size
        if rightBound > data.shape[0]:
            rightBound = data.shape[0]
        embeddings = np.concatenate((embeddings, modelSent.encode(list(data.iloc[leftBound:rightBound]['Tweet']))), axis=0)
        leftBound += batch_size
        print("Len: ", embeddings.shape)

Len:  (1000, 768)
Len:  (2000, 768)
Len:  (3000, 768)
Len:  (4000, 768)
Len:  (5000, 768)
Len:  (5350, 768)


In [54]:
lex_emb = Lexrank(embeddings)

In [55]:
lex_emb.build_graph_cosine(sim_thres = 0.78, batch_size = 1000)

In [56]:
lex_emb.train(lexrank_iter=100, damping_factor=0.85)

Iteration: 0
Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90


In [62]:
sentIds_cosine = lex_emb.extract_summary(n_sents=15, cosine_thres=0.9, max_sent=100)

Extracting sentences....
Sent scores: 5350
selected one: 3558, 0.001152850890888253
selected one: 1195, 0.0011375807186788359
Sent 810 is similar to a 3558: 0.9211936223453991
Sent 1131 is similar to a 3558: 0.912608408354179
selected one: 1152, 0.0009345352142831592
selected one: 2568, 0.000927536190769085
selected one: 2001, 0.000924811542117666
Sent 60 is similar to a 1152: 0.9434348653809289
Sent 914 is similar to a 1152: 0.9274425469712065
selected one: 2843, 0.0008490409664119972
Sent 2007 is similar to a 2001: 0.9749696544755166
Sent 1321 is similar to a 2001: 0.9068680070694313
selected one: 5274, 0.000837225491087316
selected one: 3125, 0.0008289886469287558
Sent 3185 is similar to a 3558: 0.922693606799344
Sent 2346 is similar to a 2843: 0.9011879173053515
selected one: 2979, 0.0008027681202309316
selected one: 2840, 0.0008022883185867872
selected one: 4465, 0.00080179476815103
selected one: 3329, 0.0007997973860152281
Sent 3580 is similar to a 2843: 0.910772491851759
selecte

In [63]:
#first 5000 tweets travel ban with url
for i, idx in enumerate(sentIds_cosine):
    print(i,".",idx, data.iloc[idx]['Tweet'])

0 . 3558 hurricane sandy bout to fuck errything up
1 . 1195 getting ready for hurricane sandy :) (@ frankenstorm apocalypse - hurricane sandy w/ 2656 others )
2 . 1152 i'm at frankenstorm apocalypse - hurricane sandy ( nj ) w/ 127 others
3 . 2568 tell me more about this devastating category 1 storm ... ?? #hurricane #sandy #sigh
4 . 2001 due to hurricane sandy n ! ggas be like ...
5 . 2843 ‘ frankenstorm ’ threatens eastern us - wall street closes as millions prepare for deadly hurricane sandy’s arrival an ...
6 . 5274 the fujiwhara effect explained and what it means for #hurricane #sandy as it heads to u.s. #frankenstorm
7 . 3125 news : sandy loses hurricane status & still dangerous will be referred to as superstorm sandy - eye nears landfal ...
8 . 2979 yea i heard . yall swear its gon be bad smh rt : no but we're expecting a hurricane ??
9 . 2840 hurricane sandy in the way #nsu <<<<<
10 . 4465 attn #coast2coast industry mixer #nyc edition 10/29 is canceled for #hurricane #sandy catc

In [67]:
bertscore_dict = {}
for key, value in lex_emb.graph.items():
    bertscore_dict[key] = len(value)


In [68]:
bertscore_dict = {k: v for k, v in sorted(bertscore_dict.items(), key=lambda item: item[1], reverse = True)}

In [69]:
bertscore_dict

{3558: 391,
 1195: 366,
 810: 342,
 1131: 327,
 2001: 319,
 2568: 316,
 1152: 315,
 60: 300,
 914: 298,
 2007: 296,
 1321: 288,
 5274: 283,
 2840: 280,
 3125: 271,
 3185: 270,
 2843: 261,
 2346: 259,
 3614: 255,
 3580: 254,
 3669: 253,
 4311: 253,
 933: 251,
 2344: 251,
 1476: 249,
 1353: 248,
 2053: 247,
 3776: 246,
 4465: 246,
 4596: 245,
 4768: 241,
 980: 240,
 1494: 240,
 3128: 240,
 2062: 234,
 394: 233,
 673: 231,
 993: 231,
 403: 230,
 354: 229,
 5063: 229,
 600: 228,
 1390: 228,
 1020: 228,
 3371: 228,
 2206: 227,
 3968: 226,
 1822: 224,
 3432: 222,
 4953: 222,
 758: 221,
 4240: 221,
 53: 220,
 2357: 220,
 2603: 220,
 1116: 219,
 2124: 219,
 145: 218,
 664: 216,
 631: 214,
 1660: 214,
 2851: 214,
 1931: 213,
 4995: 212,
 378: 211,
 1028: 211,
 4839: 211,
 4587: 211,
 3573: 208,
 2562: 206,
 2120: 206,
 4147: 205,
 805: 204,
 3112: 204,
 3458: 204,
 3329: 204,
 5024: 204,
 2056: 199,
 3344: 199,
 5214: 199,
 4626: 198,
 4467: 198,
 2979: 197,
 30: 195,
 3939: 194,
 5136: 194,
 2

In [71]:
count = 0
selected = []
for key, value in bertscore_dict.items():
    
    if count>0:
        added = True
        for k in selected:
            if k in lex_emb.graph[key]:
                if lex_emb.graph[key][k]>0.18:
                    added = False
                    break
        
        if added==True:
            selected.append(key)
            count+=1
            print(count, ".", key, str(data.iloc[key]['Tweet']))
    else:
        selected.append(key)
        count+=1
        print(key, str(data.iloc[key]['Tweet']))
            
    if count > 15: 
        break

3558 hurricane sandy bout to fuck errything up
2 . 4596 you've been quoted in my #storify story hurricane sandy : woburn , mass .
3 . 3329 will sandy be a hurricane at landfall def will mess up y #nhc didnt issue hurr warns
4 . 2056 nyc doe schools will be closed on monday , oct . 29 , 2012 due to projected severe weather conditions - aka hurricane sandy !
5 . 3752 hahaha that sucks . if the hurricane is bad y'all probably gonna get blown away .
6 . 2028 we are praying for those affected by hurricane sandy . may the one that even the winds and waves obey protect you in ...
7 . 5216 we're gonna be in a hurricane ! #america
8 . 5305 i hope everyone that was making jokes about hurricane sandy feels ridiculous now knowing people lost their lives du ...
9 . 2196 images of damage caused by hurricane sandy in brooklyn , ny .
10 . 4174 hurricane sandy blows election off course : washington ( reuters ) - hurricane sandy blew the u.s. presidential rac ...
11 . 4494 idk i guess cause of the hurri

# 2. Extracting bertscore

In [20]:
indices = []
with open("/home/ehoang/hnt/data/files/travel_ban_all.txt", 'r') as f:
    for line in f:
        idx = line.split(',')[0]
        indices.append(int(idx))

In [21]:
indices = sorted(indices)

In [22]:
unconsidered = [i for i in range(data.shape[0]) if i not in indices]

In [23]:
print(len(unconsidered), data.shape[0] - len(indices))

81041 81041


In [23]:
# unconsidered = np.arange(0, data.shape[0], 1)

In [24]:
new_batches = []
size = 8105
for i in range(0, len(unconsidered), size):
    if i+size > len(unconsidered):
        rightB = len(unconsidered)
    else:
        rightB = i+size
    new_batches.append([j for j in unconsidered[i:rightB]])
    

In [25]:
x = sum(len(y) for y in new_batches)
print(x, len(unconsidered), len(new_batches))

81041 81041 10


In [26]:
file = "/home/ehoang/hnt/data/files/travel_ban_all.txt"
def compute_bert(batch, device, thres):
   
    scorer = BERTScorer(lang='en', rescale_with_baseline = True, idf = True, idf_sents = list(data['Tweet']), 
                               device = 'cuda:'+str(device))
    print("device: {}..running {}".format(device, len(batch)))
    
    for idx in batch:
        # compute bert score
        batch_size = 1000
        sim_score = []
        sim_idx = []
        for i in range(idx+1, data.shape[0], batch_size):
            rightBound = i+batch_size
            if i + batch_size > data.shape[0]:
                rightBound = data.shape[0]
            sim = scorer.score([str(data.iloc[idx]['Tweet'])]*(rightBound -i), list(data.iloc[i:rightBound]['Tweet']))[0]
            sim_score+=list(sim.numpy())
#         print("len: ", len(sim_score))
        sim_score = np.array(sim_score)
        sim_idx = np.where(sim_score>thres)[0]
        sim_score = sim_score[sim_idx]
#         if len(sim_idx) == 0:
#             continue
        if (len(sim_idx) !=0):
            sim_idx = sim_idx+(idx+1)
        with open(file, 'a') as f:
            f.write("{},{},{}\n".format(idx, str(list(sim_idx)), str(list(sim_score))))
#         print("{},{},{}\n".format(idx, str(list(sim_idx)), str(list(sim_score))))


In [ ]:

n_jobs = 10
results_xx = Parallel(n_jobs = n_jobs)(delayed(compute_bert)(batch, i%2+2, 0.0) for i, batch in enumerate(new_batches))

# extracting first token embeddings bert